# Model Selection and Evaluation - Part 2

## Activity 3 - Cross Validation

In the previous activity, we made the classic mistake of using our validation data to predict our generalization error. This tends to give misleadingly optimistic predictions about how well we will do on unobserved data. Remember that we carefully picked our hyperparameter values to do as well as possible *on our held-out data*. We shouldn't be surprised when our model performs better on that data than on unobserved data. This problem is particularly acute if our data set is small.

The traditional solution is to divide our data into three disjoint sets: **training**, **validation**, and **testing**:
* The **training** set is used to fit the model.
* The **validation** set is used to evaluate models for the purpose of hyperparameter selection. 
* The **test** set is kept in a locked room guarded by jaguars. We only look at the testing set ONCE, when we have finalized our model. That way our performance on the test set gives us an unbiased estimate of our generalization error.

This traditional approach is fine if we have a lot of data to work with. If the data set is small, we are faced with a painful dilemma: More validation data means better model selection. More testing data means more accurate model evaluation. More training data means better models. Any data we use for one purpose can't be used for the others.

**Cross validation** is one way to use limited data more effectively.  The cells below walk us through an example of using cross validation for hyperparameter tuning.

In [ ]:
# We need to reimport and reload everything...
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import datasource

from sklearn.tree import DecisionTreeRegressor

# Grab our training data
source = datasource.DataSource()
X, y = source.gen_data(100, seed=100)

# Split our data into a training and testing set...
split_point = int(X.shape[0] * .8) # Use 80% of the data to train the model

X_train = X[0:split_point, :]
y_train = y[0:split_point]

X_test = X[split_point::, :] # This data will ONLY be used for final evaluation.
y_test = y[split_point::]

The following cell shows how we can use the scikit-learn `KFold` class to automatically split up our training data for k-fold cross validation. Take a minute to read through this code to make sure you understand what's going on.

In [ ]:
from sklearn.model_selection import KFold

folds = 10
max_max_leaves = 80
kf = KFold(n_splits=folds)

mses = np.zeros((folds, max_max_leaves - 2)) # (can't have 0 or 1 leaves)

# Loop over all of the hyperparameter settings
for max_leaves in range(2, max_max_leaves):
    
    k = 0
    
    # Evaluate each one K-times
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X[train_index], X[val_index]
        y_tr, y_val = y[train_index], y[val_index]

        tree = DecisionTreeRegressor(max_leaf_nodes=max_leaves)
        tree.fit(X_tr, y_tr)
        
        y_val_predict = tree.predict(X_val)
        mses[k, max_leaves - 2] = np.sum((y_val - y_val_predict)**2) / y_val.size
        
        k += 1
        
# Average across the k folds
mse_avg = np.mean(mses, axis=0)

plt.plot(np.arange(2, max_max_leaves), mse_avg)
plt.xlabel('max leaves')
plt.ylabel('MSE')
plt.show()

If we are real experts in scikit-learn, we can automate some of this by using the `cross_val_score` function:  

(There are also library routines for [automating the entire process of hyperparameter tuning](https://scikit-learn.org/stable/modules/grid_search.html).) 

In [ ]:
from sklearn.model_selection import cross_val_score

mses = np.zeros((folds,max_max_leaves - 2))

# Loop over all of the hyperparameter settings
for size in range(2, max_max_leaves):
    tree = DecisionTreeRegressor(max_leaf_nodes=size)
    
    # Returns an array of cross validation results.
    mses[:, size - 2] = -cross_val_score(tree, X_train, y_train, 
                                         cv=folds, scoring='neg_mean_squared_error')
    
mse_avg = np.mean(mses, axis=0)

plt.plot(np.arange(2, 80), mse_avg)
plt.show()
plt.xlabel('max leaves')
plt.ylabel('MSE')


### Question

* Based on the results above, what is the most promising hyperparameter value?

### Answer

* 

Now that we have a value for our hyperparameter, let's train our final model on the *full* training set and use our locked-away testing set to predict model performance:

In [ ]:
tree = DecisionTreeRegressor(max_leaf_nodes=????) # Put your best hyperparameter here!

# Train using ALL the training data
tree.fit(X_train, y_train)

# Test on held-out testing data
y_test_predict = tree.predict(X_test)
mse = np.sum((y_test - y_test_predict)**2) / y_test.size

print("Predicted MSE: {:.4f}".format(mse))

Since none of the data we are testing on here was used *in any way* to design or fit the model, this value should give us an unbiased estimate of our generalization error.  Let's try testing on some new unobserved data to see how good our estimate is:

In [ ]:

# Let's see how we do on unobserved data... 
X_new, y_new = source.gen_data(5000, seed=200)
y_new_predict = tree.predict(X_new)
mse = np.sum((y_new - y_new_predict)**2) / y_new.size
print("MSE: {:.4f}".format(mse))

### Questions

Take a look back at the results from Exercise 2 and answer the following questions:

* Did the cross validation approach improve our results in terms of model selection?  Justify your answer.
* Did maintaining a proper test set improve our results in terms of model evaluation?  Justify your answer.

### Answers

* 
* 

## Exercise 4 - Bias Variance Decomposition

Earlier in the semester we spent some time considering the bias/variance decomposition: 

$$E[(y - \hat{f}(x))^2] = (Bias[\hat{f}(x)])^2 + Variance[\hat{f}(x)] + \sigma^2$$

### Questions

* What term above corresponds to the notion of *generalization error* that we have been discussing?
* How is hyperparameter selection related to the Bias term?
* How is the training set size related to the Variance term?


### Answers

*
*
*

## Exercise 5 - Nested Cross Validation

There is still one drawback of our approach from the previous exercise:  Our testing set was quite small, which limits the reliability of our model evaluation. Maybe the test set happens to contain particularly difficult or particularly easy instances? 

In a perfect world, we would like to be able to use *all* of our data for training and validation and *all* of our data for testing.  In a way, we can accomplish this by using **nested cross validation**.  This involves generating K-fold data splits at two levels. At the top level, we iterate over Testing/Training splits for the purpose of model evaluation.  For *each* Training/Testing split we iterate over Training/Validation splits for the purpose of model tuning.  This page provides an [example of nested cross-validation using scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html). 

If you have time, go ahead and use nested cross validation to redo our example above.  Does it improve our estimate of generalization error? 